Este cuaderno es simplemente una versión anotada en Jupyter del programa CSDALindenberg.jl. 

Usa el operador convexo de Lindenberg como Laplaciano.
Para usarse requiere un archivo *jld que contenga el LFP
como fue registrado en el experimento (o promediado sobre eventos evocados),
y, optativamente, una lista de electrodos inusables ("saturados").

In [1]:
using JLD
include("LindenbergOperadores.jl")
importall LindenbergOperadores

In [16]:
arxnombre="../JLDFiles/estriadoscachos/Corte_01_estriado_slice1_1004AP.jld"
arx=load(arxnombre)

Dict{String,Any} with 6 entries:
  "Canalesrespuesta" => Set(Array{Int64,1}[[24, 64], [57, 11], [9, 28], [11, 14…
  "desde"            => 7500
  "hasta"            => 8000
  "LFPSaturados"     => [4125.0 0.0 … 2.01416 6.04248; -8.05664 -2.01416 … -12.…
  "freq"             => 17.8555
  "CanalesSaturados" => Set(Array{Int64,1}[[8, 31], [8, 40], [9, 38], [10, 40],…

In [17]:
#quitemos la terminacion de los nombres de archivo
palabra=replace(arxnombre,".jld", "")
println(palabra)

../JLDFiles/estriadoscachos/Corte_01_estriado_slice1_1004AP


In [18]:
# cargando los datos importantes
if haskey(arx, "LFPSaturados")
    LFP=arx["LFPSaturados"]
else
    LFP=arx["LFPTotal"]
end
saturados=arx["CanalesSaturados"]
respuestas=arx["Canalesrespuesta"];
frecuencia=arx["freq"]

17.85550205219098

In [19]:
lfpParchado=copy(LFP)
#Poner en cero los canales inservibles
for m in saturados
    q=m[1]
    p=m[2]
    lfpParchado[q,p,:]=0
end

In [20]:
listaredux=TiraOrillas(saturados)


Set(Any[[9, 38], [9, 41], [9, 37], [10, 41], [9, 31], [11, 41], [7, 30], [9, 40], [8, 40], [10, 40], [8, 41], [13, 11], [8, 35], [23, 63], [8, 31]])

In [21]:
#Creamos una mancha suave sobre el canal saturado.
for m in listaredux
        q=m[1]
        p=m[2]
        vecinos=vecindad8(m)
        lfpParchado[q,p,:]=promediasobreconjunto(vecinos,lfpParchado)
end


In [22]:
#en realidad nunca haces referencia al tercer numero como tiempo
(mu,nu,lu)=size(lfpParchado)
#Aplicamos un suavizado Gaussiano Temporal (esto afecta mucho las animaciones)
lfpplanchado=zeros(mu,nu,lu)
for j=1:mu,l=1:nu
    porromponpon=vec(lfpParchado[j,l,:])
    lfpplanchado[j,l,:]=GaussSuavizarTemporal(porromponpon)
end



In [23]:

aux1=zeros(mu,nu,lu)
aux2=zeros(mu,nu,lu)
#Suavizamos espacialmente con un filtro Gaussiano bidimensional el LFP.
#Posteriormente sacamos el dCSD.
for t=1:lu
    aux1[:,:,t]=GaussianSmooth(lfpplanchado[:,:,t])
    aux2[:,:,t]=DiscreteLaplacian(aux1[:,:,t])
end
CSD=-aux2;


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Float64}, ::Tuple{Int64,Int64}) at ./deprecated.jl:57
 [3] DiscreteLaplacian(::Array{Float64,2}) at /home/karel/RGutWork/CSDA/LindenbergOperadores.jl:137
 [4] macro expansion at ./In[23]:8 [inlined]
 [5] anonymous at ./<missing>:?
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/karel/.julia/v0.6/IJulia/src/execute_request.jl:193
 [7] (::Compat.#inner#14{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /home/karel/.julia/v0.6/Compat/src/Compat.jl:332
 [8] eventloop(::ZMQ.Socket) at /home/karel/.julia/v0.6/IJulia/src/eventloop.jl:8
 [9] (::IJulia.##13#16)() at ./task.jl:335
while loading In[23], in expression starting on line 6


In [25]:

#Descartamos variables auxiliares
lfpParchado=0
aux1=0
aux2=0

#Observa que vas a guardar en el mismo archivo todo. 
paguardar=load(arxnombre)
paguardar["CSDLindenberg"]=CSD
save(arxnombre,paguardar)

